In [ ]:
# importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
data = pd.read_csv("../Dataset/All_Anime.csv")
data.head()

In [ ]:
# analyse the data
data.info()


In [ ]:
rows, columns = data.shape
print("number of rows = ",rows)
print("number of columns = ",columns)


In [ ]:
print("Ratings Summary:")
print(df['Rating'].describe(), '\n')

In [ ]:
#Handle time series data
def year_processing(year):
    try:
        if isinstance(year, str):
            if '-' in year:  # Handle ranges like "2013-2023"
                start_year, end_year = year.split('-')
                return (int(start_year) + int(end_year)) // 2
            elif year.isdigit():  # Handle single years like "2000"
                return int(year)
        return np.nan  # Return NaN for invalid or missing values
    except:
        return np.nan  # Catch any errors and return NaN

data['Year'] = data['Year'].apply(year_processing)

# Handle the "Duration" column
def duration_processing(duration):
    try:
        if isinstance(duration, str) and 'min' in duration:
            return int(duration.split()[0])  # Extract the number before "min"
        return np.nan  # Return NaN for invalid or missing values
    except:
        return np.nan  # Catch any errors and return NaN

data['Duration'] = data['Duration'].apply(duration_processing)

# Display the processed data
print(data[['Year', 'Duration']].head())

In [ ]:
# check for missing values and handle them
# Drop the 'Unnamed: 0' 
data.drop(data.columns[0], axis=1, inplace = True)

# Fill missing 'Year' with the most frequent value (mode) 
data['Year'] = data['Year'].fillna(data['Year'].mode()[0])

# Fill missing 'Duration' with the most frequent values
data['Duration'] = data['Duration'].fillna(data['Duration'].mode()[0])


# Fill missing 'Certificate' with the mode
data['Certificate'] = data['Certificate'].fillna(data['Certificate'].mode()[0])

# Fill missing 'Genre' with the mode
data['Genre'] = data['Genre'].fillna(data['Genre'].mode()[0])

# Fill missing 'Rating' with the median rating
data['Rating'] = data['Rating'].fillna(data['Rating'].median())

# Verify that there are no missing values left
print(data.isnull().sum())

In [ ]:
# use word2vec for "story" column
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
# Visualise the rating pattern over the years using boxplot

In [ ]:
# Analyse how the duration affects the rating

In [ ]:
# Standardise numerical values and handle categorical columns (get dummies for the genre)

In [ ]:
# implement linear regression model

In [ ]:
# implement random forest regressor

In [ ]:
 # evaluate both models

In [ ]:
# compare the models